In [1]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import json
import numpy as np

In [2]:
with open("round_robin/DR_FSP_DR_SP.json") as f:
    logs = json.load(f)

In [81]:
def plot_performances(agent_name: str, experiment: str):
    with open(f"round_robin/{experiment}.json") as f:
        logs = json.load(f)
        df = {
            int(key): {
                "avg": np.array(logs[agent_name][key]).mean(),
                "std": np.array(logs[agent_name][key]).std(),
            }
            for key in sorted(
                logs[agent_name].keys(),
            )
        }
    df = pd.DataFrame(df.values(), index=df.keys(), columns=["avg", "std"])
    df["lower"] = df["avg"] - df["std"]
    df["upper"] = df["avg"] + df["std"]

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df["avg"],
            mode="lines",
            line=dict(color="blue"),
            name="Average",
        )
    )

    fig.add_trace(
        go.Scatter(
            x=np.array([df.index, df.index[::-1]]).flatten(),
            y=np.array([df["upper"], df["lower"][::-1]]).flatten(),
            fill="toself",
            fillcolor="rgba(0, 0, 255, 0.2)",
            line=dict(color="rgba(255, 255, 255, 0)"),
            name="Error Band",
        )
    )

    fig.update_layout(
        title=agent_name,
        xaxis_title="Checkpoints",
        yaxis_title="Returns",
        showlegend=True,
    )
    fig.show()


plot_performances("DR_FSP", "DR_FSP_DR_SP")

In [76]:
n = 3 # n_curriculums 
C = 40 # n_checkpoints 
S = 5 # n_seeds
E = 5 # n_episodes_per_matchup 
total_episodes = n*C*S**2*E
total_episodes

15000